In [9]:
!pip install pandas matplotlib seaborn numpy scikit-learn scipy pgmpy networkx

  Obtaining dependency information for pgmpy from https://files.pythonhosted.org/packages/c7/e6/e451590c2341b3d59d7b613e1af80daefd9e2873f7c9ad3d498ad84e7f44/pgmpy-0.1.26-py3-none-any.whl.metadata
  Obtaining dependency information for networkx from https://files.pythonhosted.org/packages/b9/54/dd730b32ea14ea797530a4479b2ed46a6fb250f682a9cfb997e968bf0261/networkx-3.4.2-py3-none-any.whl.metadata
  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/11/c5/2370d96b31eb1841c3a0883a492c15278a6718ccad61bb6a649c80d1d9eb/torch-2.6.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for statsmodels from https://files.pythonhosted.org/packages/4b/e4/f9e96896278308e17dfd4f60a84826c48117674c980234ee38f59ab28a12/statsmodels-0.14.4-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/d0/30/dc54f88dd4a2b5dc8a0279bdd7270e735851848b762aeb1c1184ed1f6b14/tqdm-4.67.1-py3-none-any.w


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
import scipy.stats as stats
import sklearn
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from pgmpy.estimators import PC, HillClimbSearch, BicScore
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.sampling import BayesianModelSampling
import seaborn as sns

C:\Users\franb\PycharmProjects\traffic-forecasting\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
metrla = pd.read_csv("./METR-LA.csv")
pemsbay = pd.read_csv("./PEMS-BAY.csv")
pemsbay_meta = pd.read_csv("./PEMS-BAY-META.csv")

In [7]:
pemsbay_meta

,sensor_id,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude,Length,Type,Lanes,Name,User_ID_1,User_ID_2,User_ID_3,User_ID_4
0,400649,17,N,4,85,44112.0,9.06,21.613,37.251741,-121.958118,0.850,ML,2,LARK AVE,DT185,L2-N-37-017-00906,998009,417770
1,400715,17,N,4,85,44112.0,9.4,21.953,37.256205,-121.955355,0.245,ML,2,SB 85 rm-n-conn,DT244,L2-N-37-017-00940,998013,417774
2,400240,17,N,4,85,44112.0,9.55,22.103,37.258184,-121.953853,0.535,ML,3,NB 85 rm-n-conn,DT260,L3-N-37-017-00955,998017,420767
3,403329,17,N,4,85,10340.0,10.47,23.023,37.270267,-121.947353,0.510,ML,3,Camden ave rm-n-loop,DTB25,L3-N-37-017-01047,997278,430594
4,400436,17,N,4,85,10340.0,10.57,23.123,37.271687,-121.947037,0.905,ML,3,Camden Ave rm-n-diag,DT267,l3-N-37-017-01057,998021,420772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,400973,237,E,4,85,69084.0,6.18,6.237,37.417801,-121.972288,0.973,ML,3,GREAT AMERICA PKWY,DT852,L3-E-37-237-00618,998245,420562
321,400567,237,E,4,85,68000.0,6.7,6.752,37.418931,-121.962821,0.462,ML,4,N First St,DT193,L4-E-37-237-00670,998232,420414
322,400104,237,E,4,85,68000.0,7.11,7.162,37.419313,-121.955480,0.665,ML,3,NORTH FIRST STREET,DT814,L3-E-37-237-00711,998233,420567
323,400296,237,E,4,85,68000.0,8.03,8.082,37.420359,-121.939150,0.530,ML,3,oppo Zanker Rd rm-w-loop,DT822,L3-E-37-237-00803,998236,420572


In [13]:
pemsbay

,Unnamed: 0,400001,400017,400030,400040,400045,400052,400057,400059,400065,...,409525,409526,409528,409529,413026,413845,413877,413878,414284,414694
0,2017-01-01 00:00:00,71.4,67.8,70.5,67.4,68.8,66.6,66.8,68.0,66.8,...,68.8,67.9,68.8,68.0,69.2,68.9,70.4,68.8,71.1,68.0
1,2017-01-01 00:05:00,71.6,67.5,70.6,67.5,68.7,66.6,66.8,67.8,66.5,...,68.4,67.3,68.4,67.6,70.4,68.8,70.1,68.4,70.8,67.4
2,2017-01-01 00:10:00,71.6,67.6,70.2,67.4,68.7,66.1,66.8,67.8,66.2,...,68.4,67.4,68.4,67.5,70.2,68.3,69.8,68.4,70.5,67.9
3,2017-01-01 00:15:00,71.1,67.5,70.3,68.0,68.5,66.7,66.6,67.7,65.9,...,68.5,67.5,68.5,67.5,70.4,68.7,70.2,68.4,70.8,67.6
4,2017-01-01 00:20:00,71.7,67.8,70.2,68.1,68.4,66.9,66.1,67.7,66.1,...,68.5,67.7,68.5,67.4,69.6,69.1,70.0,68.4,71.0,67.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52111,2017-06-30 23:35:00,70.9,66.3,68.4,61.0,61.8,66.9,66.0,67.4,65.4,...,64.6,66.8,64.7,61.1,68.4,61.4,70.5,68.2,71.6,66.2
52112,2017-06-30 23:40:00,71.3,66.6,68.7,60.9,62.0,66.2,66.5,68.2,64.9,...,65.1,67.3,64.8,60.8,69.8,62.2,69.4,68.6,71.6,68.4
52113,2017-06-30 23:45:00,71.4,66.9,68.1,61.1,62.0,66.7,65.8,68.2,64.2,...,64.9,66.5,64.6,61.3,69.5,63.8,70.6,68.4,71.6,66.6
52114,2017-06-30 23:50:00,72.2,66.5,68.0,61.1,62.5,65.8,67.1,68.3,64.0,...,64.1,65.9,64.8,61.3,69.6,64.2,70.2,68.7,71.6,68.4


In [21]:
# Step 1: Load and prepare the data
def load_and_prepare_data(file_path):
    # Load the data
    df = pd.read_csv(file_path)
    df.drop("Unnamed: 0", inplace=True, axis=1)
    
    # Display basic information about the dataset
    print(f"Dataset shape: {df.shape}")
    print("\nFirst few rows of the dataset:")
    print(df.head())
    
    # Check for missing values
    print("\nMissing values in each column:")
    print(df.isnull().sum())
    
    # Basic statistics
    print("\nBasic statistics:")
    print(df.describe())
    
    return df

# Step 2: Preprocess data for causal discovery
def preprocess_for_causal_discovery(df, sample_size=1000, discretize=True, n_bins=3):
    """
    Preprocess the data for causal discovery:
    - Sample data if it's too large
    - Optionally discretize continuous variables
    - Handle missing values
    """
    # Sample data if it's too large
    if len(df) > sample_size:
        df_sample = df.sample(sample_size, random_state=42)
    else:
        df_sample = df.copy()
    
    # Fill missing values
    df_sample = df_sample.fillna(df_sample.mean())
    
    # Optionally discretize continuous variables
    if discretize:
        for col in df_sample.select_dtypes(include=[np.number]).columns:
            df_sample[col] = pd.qcut(df_sample[col], n_bins, labels=False, duplicates='drop')
    
    return df_sample

# Step 3: Estimate causal structure
def estimate_causal_structure(df, method='pc'):
    """
    Estimate the causal structure using either PC algorithm or score-based approach
    """
    if method == 'pc':
        # PC algorithm (constraint-based)
        est = PC(data=df)
        causal_graph = est.estimate(variant='stable', ci_test='chi_square', significance_level=0.05)
        
    elif method == 'score':
        # Score-based approach using BIC score
        hc = HillClimbSearch(data=df)
        bic = BicScore(data=df)
        causal_graph = hc.estimate(scoring_method=bic)
    
    return causal_graph

# Step 4: Visualize the causal graph
def visualize_causal_graph(causal_graph, title="Estimated Causal Graph"):
    """
    Visualize the causal graph
    """
    plt.figure(figsize=(12, 10))
    
    # Create a directed graph from the causal graph
    G = nx.DiGraph()
    
    # Add edges
    if hasattr(causal_graph, 'edges'):
        edges = causal_graph.edges()
    else:
        edges = causal_graph.edges
    
    for edge in edges:
        G.add_edge(edge[0], edge[1])
    
    # Draw the graph
    pos = nx.spring_layout(G, seed=42)
    nx.draw(G, pos, with_labels=True, node_color='lightblue', 
            node_size=3000, arrowsize=20, font_size=12,
            font_weight='bold', edge_color='gray')
    
    plt.title(title, fontsize=16)
    plt.tight_layout()
    plt.show()
    
    return G

# Step 5: Fit Bayesian Network parameters
def fit_bayesian_network(df, structure):
    """
    Fit parameters of the Bayesian Network given the structure
    """
    model = BayesianNetwork(structure)
    model.fit(df)
    return model

# Step 6: Validate and interpret results
def validate_model(model, df, test_cols):
    """
    Validate the model by sampling and comparing distributions
    """
    # Sample from the model
    inference = BayesianModelSampling(model)
    samples = inference.forward_sample(size=len(df))
    
    # Compare distributions of original vs. sampled data
    plt.figure(figsize=(15, 5*len(test_cols)))
    
    for i, col in enumerate(test_cols):
        plt.subplot(len(test_cols), 2, 2*i+1)
        sns.histplot(df[col], kde=True)
        plt.title(f"Original Distribution: {col}")
        
        plt.subplot(len(test_cols), 2, 2*i+2)
        sns.histplot(samples[col], kde=True)
        plt.title(f"Sampled Distribution: {col}")
    
    plt.tight_layout()
    plt.show()

# Step 7: Find potential causal relationships
def identify_causal_relationships(G, df):
    """
    Identify and analyze potential causal relationships
    """
    # Find nodes with highest out-degree (potential causes)
    out_degrees = dict(G.out_degree())
    influential_nodes = sorted(out_degrees.items(), key=lambda x: x[1], reverse=True)[:5]
    
    print("Most influential nodes (potential causes):")
    for node, degree in influential_nodes:
        if degree > 0:
            print(f"Node: {node}, Influences: {degree} other nodes")
            successors = list(G.successors(node))
            print(f"  Directly influences: {successors}")
            
            # Calculate correlation
            if len(successors) > 0:
                corr_values = []
                for succ in successors:
                    if node in df.columns and succ in df.columns:
                        corr = df[node].corr(df[succ])
                        corr_values.append((succ, corr))
                        print(f"  Correlation with {succ}: {corr:.4f}")
                
                # Visualize correlations
                if corr_values:
                    plt.figure(figsize=(10, 6))
                    nodes, corrs = zip(*corr_values)
                    plt.bar(nodes, corrs)
                    plt.axhline(y=0, color='r', linestyle='-')
                    plt.title(f"Correlation of {node} with influenced nodes")
                    plt.ylabel("Correlation coefficient")
                    plt.xticks(rotation=45)
                    plt.tight_layout()
                    plt.show()

# Main function to run all steps
def build_traffic_causal_graph(file_path, method='pc', discretize=True, n_bins=3):
    print("Step 1: Loading and preparing data...")
    df = load_and_prepare_data(file_path)
    
    print("\nStep 2: Preprocessing data...")
    processed_df = preprocess_for_causal_discovery(df, discretize=discretize, n_bins=n_bins)
    
    print("\nStep 3: Estimating causal structure...")
    causal_graph = estimate_causal_structure(processed_df, method=method)
    
    print("\nStep 4: Visualizing causal graph...")
    G = visualize_causal_graph(causal_graph)
    
    print("\nStep 5: Fitting Bayesian Network...")
    bayesian_model = fit_bayesian_network(processed_df, list(G.edges()))
    
    print("\nStep 6: Validating model...")
    # Select a few important columns for validation
    test_cols = list(processed_df.columns[:3])  # Using first 3 columns for demonstration
    validate_model(bayesian_model, processed_df, test_cols)
    
    print("\nStep 7: Identifying causal relationships...")
    identify_causal_relationships(G, processed_df)
    
    return {
        'data': df,
        'processed_data': processed_df,
        'causal_graph': G,
        'bayesian_model': bayesian_model
    }

In [ ]:
# Simply run this function with your data file
results = build_traffic_causal_graph("./PEMS-BAY.csv")

Step 1: Loading and preparing data...
Dataset shape: (52116, 325)

First few rows of the dataset:
   400001  400017  400030  400040  400045  400052  400057  400059  400065  \
0    71.4    67.8    70.5    67.4    68.8    66.6    66.8    68.0    66.8   
1    71.6    67.5    70.6    67.5    68.7    66.6    66.8    67.8    66.5   
2    71.6    67.6    70.2    67.4    68.7    66.1    66.8    67.8    66.2   
3    71.1    67.5    70.3    68.0    68.5    66.7    66.6    67.7    65.9   
4    71.7    67.8    70.2    68.1    68.4    66.9    66.1    67.7    66.1   

   400069  ...  409525  409526  409528  409529  413026  413845  413877  \
0    69.0  ...    68.8    67.9    68.8    68.0    69.2    68.9    70.4   
1    68.2  ...    68.4    67.3    68.4    67.6    70.4    68.8    70.1   
2    67.8  ...    68.4    67.4    68.4    67.5    70.2    68.3    69.8   
3    67.8  ...    68.5    67.5    68.5    67.5    70.4    68.7    70.2   
4    67.8  ...    68.5    67.7    68.5    67.4    69.6    69.1    70.

Working for n conditional variables: 1:  20%|██        | 1/5 [01:47<07:09, 107.29s/it]